In [1]:
%matplotlib qt

import matplotlib.pyplot as plt
from poisson_fem import PoissonFEM
import ROM
import GenerativeSurrogate as gs
import Data as dta
import numpy as np
import scipy.sparse as sps
import scipy.sparse.linalg as lg
import time
import petsc4py
import sys
petsc4py.init(sys.argv)
from petsc4py import PETSc
import torch
from torch import optim

/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/c

In [2]:
# Some fixed parameters
lin_dim_rom = 4                      # Linear number of rom elements
a = np.array([1, 2, 3])              # Boundary condition function coefficients
dim_z = 30                            # Latent space dimension
dtype = torch.float                  # Tensor data type

In [3]:
# Define mesh and boundary conditions
mesh = PoissonFEM.RectangularMesh(np.ones(lin_dim_rom)/lin_dim_rom)
# mesh.plot()

def origin(x):
    return np.abs(x[0]) < np.finfo(float).eps and np.abs(x[1]) < np.finfo(float).eps

def essBoundaryFun(x):
    return 0.0
mesh.setEssentialBoundary(origin, essBoundaryFun)

def domainBoundary(x):
    # unit square
    return np.abs(x[0]) < np.finfo(float).eps or np.abs(x[1]) < np.finfo(float).eps or \
            np.abs(x[0]) > 1.0 - np.finfo(float).eps or np.abs(x[1]) > 1.0 - np.finfo(float).eps
mesh.setNaturalBoundary(domainBoundary)

def flux(x):
    q = np.array([a[0] + a[2]*x[1], a[1] + a[2]*x[0]])
    return q

In [4]:
#Spepify right hand side and stiffness matrix
#Define boundary flux field
rhs = PoissonFEM.RightHandSide(mesh)
rhs.setNaturalRHS(mesh, flux)
funSpace = PoissonFEM.FunctionSpace(mesh)
K = PoissonFEM.StiffnessMatrix(mesh, funSpace)
rhs.setRhsStencil(mesh, K)

In [5]:
# Set up solver
ksp = PETSc.KSP().create()
ksp.setType('preonly')
precond = ksp.getPC()
precond.setType('cholesky')
ksp.setFromOptions() #???

In [6]:
# define rom
rom = ROM.ROM(mesh, K, rhs, ksp)

In [7]:
trainingData = dta.StokesData(range(2048))
trainingData.readData(['IMG'])
# trainingData.plotMicrostruct(1)
trainingData.reshapeInputImg()

In [8]:
model = gs.GenerativeSurrogate(rom, trainingData, dim_z)

In [ ]:
steps = int(1000)
for s in range(steps):
    print('step = ', s)
    batchSamples_pf = torch.multinomial(torch.ones(trainingData.nSamplesIn), model.batchSizeN_pf)
    batchSamples_pc = torch.multinomial(torch.ones(trainingData.nSamplesOut), model.batchSizeN_pc)
    model.optLatentDistStep()
    model.pfStep(batchSamples_pf)
    model.pcStep(batchSamples_pc)
    

step =  0
loss_pf =  tensor(11631629., grad_fn=<SubBackward0>)
loss_pc =  tensor(0.4331, grad_fn=<DotBackward>)
step =  1
loss_pf =  tensor(10922345., grad_fn=<SubBackward0>)
loss_pc =  tensor(13.6645, grad_fn=<DotBackward>)
step =  2
loss_pf =  tensor(10847678., grad_fn=<SubBackward0>)
loss_pc =  tensor(6.3698, grad_fn=<DotBackward>)
step =  3
loss_pf =  tensor(10689417., grad_fn=<SubBackward0>)
loss_pc =  tensor(3.4833, grad_fn=<DotBackward>)
step =  4
loss_pf =  tensor(10540688., grad_fn=<SubBackward0>)
loss_pc =  tensor(2.3425, grad_fn=<DotBackward>)
step =  5
loss_pf =  tensor(10480070., grad_fn=<SubBackward0>)
loss_pc =  tensor(2.1027, grad_fn=<DotBackward>)
step =  6
loss_pf =  tensor(10836524., grad_fn=<SubBackward0>)
loss_pc =  tensor(25.1408, grad_fn=<DotBackward>)
step =  7
loss_pf =  tensor(10344600., grad_fn=<SubBackward0>)
loss_pc =  tensor(24.6011, grad_fn=<DotBackward>)
step =  8
loss_pf =  tensor(10290627., grad_fn=<SubBackward0>)
loss_pc =  tensor(1.2189, grad_fn=<Dot

loss_pf =  tensor(7010813., grad_fn=<SubBackward0>)
loss_pc =  tensor(3.6756, grad_fn=<DotBackward>)
step =  74
loss_pf =  tensor(7152637.5000, grad_fn=<SubBackward0>)
loss_pc =  tensor(4.5311, grad_fn=<DotBackward>)
step =  75
loss_pf =  tensor(7069683., grad_fn=<SubBackward0>)
loss_pc =  tensor(5.0169, grad_fn=<DotBackward>)
step =  76
loss_pf =  tensor(6887920.5000, grad_fn=<SubBackward0>)
loss_pc =  tensor(4.1050, grad_fn=<DotBackward>)
step =  77
loss_pf =  tensor(6728373., grad_fn=<SubBackward0>)
loss_pc =  tensor(3.4174, grad_fn=<DotBackward>)
step =  78
loss_pf =  tensor(6661887., grad_fn=<SubBackward0>)
loss_pc =  tensor(2.0621, grad_fn=<DotBackward>)
step =  79
loss_pf =  tensor(6562073., grad_fn=<SubBackward0>)
loss_pc =  tensor(1.6378, grad_fn=<DotBackward>)
step =  80
loss_pf =  tensor(6810348., grad_fn=<SubBackward0>)
loss_pc =  tensor(1.3057, grad_fn=<DotBackward>)
step =  81
loss_pf =  tensor(6644259., grad_fn=<SubBackward0>)
loss_pc =  tensor(1.1621, grad_fn=<DotBackwa

loss_pc =  tensor(2.9852, grad_fn=<DotBackward>)
step =  146
loss_pf =  tensor(4836058.5000, grad_fn=<SubBackward0>)
loss_pc =  tensor(2.8009, grad_fn=<DotBackward>)
step =  147
loss_pf =  tensor(4809936., grad_fn=<SubBackward0>)
loss_pc =  tensor(2.9259, grad_fn=<DotBackward>)
step =  148
loss_pf =  tensor(5037110., grad_fn=<SubBackward0>)
loss_pc =  tensor(2.5416, grad_fn=<DotBackward>)
step =  149
loss_pf =  tensor(4532187., grad_fn=<SubBackward0>)
loss_pc =  tensor(2.7678, grad_fn=<DotBackward>)
step =  150
loss_pf =  tensor(4763558., grad_fn=<SubBackward0>)
loss_pc =  tensor(2.3930, grad_fn=<DotBackward>)
step =  151
loss_pf =  tensor(4600295.5000, grad_fn=<SubBackward0>)
loss_pc =  tensor(2.0808, grad_fn=<DotBackward>)
step =  152
loss_pf =  tensor(4535979., grad_fn=<SubBackward0>)
loss_pc =  tensor(2.3258, grad_fn=<DotBackward>)
step =  153
loss_pf =  tensor(4562723., grad_fn=<SubBackward0>)
loss_pc =  tensor(2.9930, grad_fn=<DotBackward>)
step =  154
loss_pf =  tensor(4468865.,

loss_pc =  tensor(4.7488, grad_fn=<DotBackward>)
step =  218
loss_pf =  tensor(3971583.5000, grad_fn=<SubBackward0>)
loss_pc =  tensor(5.1794, grad_fn=<DotBackward>)
step =  219
loss_pf =  tensor(4373474., grad_fn=<SubBackward0>)
loss_pc =  tensor(5.9284, grad_fn=<DotBackward>)
step =  220
loss_pf =  tensor(4380168., grad_fn=<SubBackward0>)
loss_pc =  tensor(7.2782, grad_fn=<DotBackward>)
step =  221
loss_pf =  tensor(4482258., grad_fn=<SubBackward0>)
loss_pc =  tensor(9.5565, grad_fn=<DotBackward>)
step =  222
loss_pf =  tensor(4937345.5000, grad_fn=<SubBackward0>)
loss_pc =  tensor(10.4378, grad_fn=<DotBackward>)
step =  223
loss_pf =  tensor(4285781., grad_fn=<SubBackward0>)
loss_pc =  tensor(9.9886, grad_fn=<DotBackward>)
step =  224
loss_pf =  tensor(4602187., grad_fn=<SubBackward0>)
loss_pc =  tensor(8.5600, grad_fn=<DotBackward>)
step =  225
loss_pf =  tensor(4747888.5000, grad_fn=<SubBackward0>)
loss_pc =  tensor(7.2768, grad_fn=<DotBackward>)
step =  226
loss_pf =  tensor(4704

loss_pf =  tensor(3642521.5000, grad_fn=<SubBackward0>)
loss_pc =  tensor(8.5923, grad_fn=<DotBackward>)
step =  290
loss_pf =  tensor(4610995., grad_fn=<SubBackward0>)
loss_pc =  tensor(16.0095, grad_fn=<DotBackward>)
step =  291
loss_pf =  tensor(4121476.2500, grad_fn=<SubBackward0>)
loss_pc =  tensor(15.2908, grad_fn=<DotBackward>)
step =  292
loss_pf =  tensor(5159740., grad_fn=<SubBackward0>)
loss_pc =  tensor(19.9908, grad_fn=<DotBackward>)
step =  293
loss_pf =  tensor(3892110.2500, grad_fn=<SubBackward0>)
loss_pc =  tensor(12.1170, grad_fn=<DotBackward>)
step =  294
loss_pf =  tensor(4875648., grad_fn=<SubBackward0>)
loss_pc =  tensor(8.8046, grad_fn=<DotBackward>)
step =  295
loss_pf =  tensor(3983895., grad_fn=<SubBackward0>)
loss_pc =  tensor(4.5501, grad_fn=<DotBackward>)
step =  296
loss_pf =  tensor(4225967., grad_fn=<SubBackward0>)
loss_pc =  tensor(4.6367, grad_fn=<DotBackward>)
step =  297
loss_pf =  tensor(4232307.5000, grad_fn=<SubBackward0>)
loss_pc =  tensor(4.8827

...

loss_pc =  tensor(1.4836, grad_fn=<DotBackward>)
step =  503
loss_pf =  tensor(1844973.5000, grad_fn=<SubBackward0>)
loss_pc =  tensor(1.4748, grad_fn=<DotBackward>)
step =  504
loss_pf =  tensor(1824289.5000, grad_fn=<SubBackward0>)
loss_pc =  tensor(1.7381, grad_fn=<DotBackward>)
step =  505
loss_pf =  tensor(1653779.5000, grad_fn=<SubBackward0>)
loss_pc =  tensor(2.0442, grad_fn=<DotBackward>)
step =  506
loss_pf =  tensor(1953500.2500, grad_fn=<SubBackward0>)
loss_pc =  tensor(1.9772, grad_fn=<DotBackward>)
step =  507
loss_pf =  tensor(1888586., grad_fn=<SubBackward0>)
loss_pc =  tensor(2.0405, grad_fn=<DotBackward>)
step =  508
loss_pf =  tensor(2128053.2500, grad_fn=<SubBackward0>)
loss_pc =  tensor(2.3741, grad_fn=<DotBackward>)
step =  509
loss_pf =  tensor(1739236.7500, grad_fn=<SubBackward0>)
loss_pc =  tensor(1.8703, grad_fn=<DotBackward>)
step =  510
loss_pf =  tensor(1811889.5000, grad_fn=<SubBackward0>)
loss_pc =  tensor(1.7411, grad_fn=<DotBackward>)
step =  511
loss_pf

loss_pc =  tensor(1.6819, grad_fn=<DotBackward>)
step =  574
loss_pf =  tensor(2150046.5000, grad_fn=<SubBackward0>)
loss_pc =  tensor(1.8442, grad_fn=<DotBackward>)
step =  575
loss_pf =  tensor(2082984., grad_fn=<SubBackward0>)
loss_pc =  tensor(1.4956, grad_fn=<DotBackward>)
step =  576
loss_pf =  tensor(1847446.2500, grad_fn=<SubBackward0>)
loss_pc =  tensor(1.5763, grad_fn=<DotBackward>)
step =  577
loss_pf =  tensor(1974538.2500, grad_fn=<SubBackward0>)
loss_pc =  tensor(1.7167, grad_fn=<DotBackward>)
step =  578
loss_pf =  tensor(2163883.2500, grad_fn=<SubBackward0>)
loss_pc =  tensor(1.6675, grad_fn=<DotBackward>)
step =  579
loss_pf =  tensor(1624888.5000, grad_fn=<SubBackward0>)
loss_pc =  tensor(1.4233, grad_fn=<DotBackward>)
step =  580
loss_pf =  tensor(1884499.7500, grad_fn=<SubBackward0>)
loss_pc =  tensor(1.5719, grad_fn=<DotBackward>)
step =  581
loss_pf =  tensor(1590752.8750, grad_fn=<SubBackward0>)
loss_pc =  tensor(1.5296, grad_fn=<DotBackward>)
step =  582
loss_pf

loss_pc =  tensor(10.7530, grad_fn=<DotBackward>)
step =  645
loss_pf =  tensor(6297200., grad_fn=<SubBackward0>)
loss_pc =  tensor(5.8419, grad_fn=<DotBackward>)
step =  646
loss_pf =  tensor(7031905., grad_fn=<SubBackward0>)
loss_pc =  tensor(7.9554, grad_fn=<DotBackward>)
step =  647
loss_pf =  tensor(7058641., grad_fn=<SubBackward0>)
loss_pc =  tensor(11.9346, grad_fn=<DotBackward>)
step =  648
loss_pf =  tensor(7872949.5000, grad_fn=<SubBackward0>)
loss_pc =  tensor(20.0658, grad_fn=<DotBackward>)
step =  649
loss_pf =  tensor(7524569., grad_fn=<SubBackward0>)
loss_pc =  tensor(25.2343, grad_fn=<DotBackward>)
step =  650
loss_pf =  tensor(9091829., grad_fn=<SubBackward0>)
loss_pc =  tensor(46.1211, grad_fn=<DotBackward>)
step =  651
loss_pf =  tensor(7897878.5000, grad_fn=<SubBackward0>)
loss_pc =  tensor(42.4998, grad_fn=<DotBackward>)
step =  652
loss_pf =  tensor(9169933., grad_fn=<SubBackward0>)
loss_pc =  tensor(56.2754, grad_fn=<DotBackward>)
step =  653
loss_pf =  tensor(74

loss_pf =  tensor(1501470.5000, grad_fn=<SubBackward0>)
loss_pc =  tensor(2.4068, grad_fn=<DotBackward>)
step =  717
loss_pf =  tensor(1584984.6250, grad_fn=<SubBackward0>)
loss_pc =  tensor(2.3644, grad_fn=<DotBackward>)
step =  718
loss_pf =  tensor(1690943.7500, grad_fn=<SubBackward0>)
loss_pc =  tensor(2.5668, grad_fn=<DotBackward>)
step =  719
loss_pf =  tensor(1763316.8750, grad_fn=<SubBackward0>)
loss_pc =  tensor(2.3462, grad_fn=<DotBackward>)
step =  720
loss_pf =  tensor(1507454.2500, grad_fn=<SubBackward0>)
loss_pc =  tensor(2.3160, grad_fn=<DotBackward>)
step =  721
loss_pf =  tensor(1309657.6250, grad_fn=<SubBackward0>)
loss_pc =  tensor(2.3794, grad_fn=<DotBackward>)
step =  722
loss_pf =  tensor(1126659.7500, grad_fn=<SubBackward0>)
loss_pc =  tensor(2.6067, grad_fn=<DotBackward>)
step =  723
loss_pf =  tensor(1279913.5000, grad_fn=<SubBackward0>)
loss_pc =  tensor(2.8600, grad_fn=<DotBackward>)
step =  724
loss_pf =  tensor(1296524.7500, grad_fn=<SubBackward0>)
loss_pc 

loss_pc =  tensor(3.2651, grad_fn=<DotBackward>)
step =  787
loss_pf =  tensor(1209271.2500, grad_fn=<SubBackward0>)
loss_pc =  tensor(3.1792, grad_fn=<DotBackward>)
step =  788
loss_pf =  tensor(1283375.5000, grad_fn=<SubBackward0>)
loss_pc =  tensor(3.2330, grad_fn=<DotBackward>)
step =  789
loss_pf =  tensor(1157871.2500, grad_fn=<SubBackward0>)
loss_pc =  tensor(4.0893, grad_fn=<DotBackward>)
step =  790
loss_pf =  tensor(1349102.3750, grad_fn=<SubBackward0>)
loss_pc =  tensor(4.0366, grad_fn=<DotBackward>)
step =  791
loss_pf =  tensor(1108545., grad_fn=<SubBackward0>)
loss_pc =  tensor(4.2486, grad_fn=<DotBackward>)
step =  792
loss_pf =  tensor(1221763.5000, grad_fn=<SubBackward0>)
loss_pc =  tensor(3.7971, grad_fn=<DotBackward>)
step =  793
loss_pf =  tensor(997515.4375, grad_fn=<SubBackward0>)
loss_pc =  tensor(3.6936, grad_fn=<DotBackward>)
step =  794
loss_pf =  tensor(1101700., grad_fn=<SubBackward0>)
loss_pc =  tensor(3.4442, grad_fn=<DotBackward>)
step =  795
loss_pf =  t

In [ ]:
model.plotInputReconstruction()
f = plt.gcf()
f.suptitle('Untrained, N = 1184', fontsize=32, y=.7)